In [ ]:
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel, GenerationConfig
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import warnings
import time

# Initialize tools
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Do this: pip install google-cloud-bigquery-storage and db_dtypes
# Suppress warnings with a specific message
warnings.filterwarnings(
    "ignore", 
    message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project."
)

In [ ]:
def generate_response(prompt, model_name="gemini-2.0-flash", temperature=0.4, top_k=30, top_p=0.3):
    ai_client = GenerativeModel(
        model_name=model_name,
        generation_config=GenerationConfig(
            temperature=temperature,  
            top_k=top_k,  
            top_p=top_p
        )
    )
    response_result = ai_client.generate_content(prompt)
    return response_result.candidates[0].content.text

In [ ]:
# Setup

project_name = "snowplow-cto-office"
bigquery_client = bigquery.Client(project=project_name)
product = "smartbill"
#product = "vnet_autopay"
#product = "eplus"
dataset_id = f"""snowplow-cto-office.snowplow_{product}"""
#full_responses_table_name = f"""snowplow-cto-office.snowplow_{product}.Vismanet_AutoPay_events_filtered_L60D"""

table_names = [f"{product}_simple_urlpath_sequences", f"{product}_events_custom_T60D", f"{product}_page_views_aggregated", f"{product}_session_chunks", f"{product}_simple_title_sequences"]

In [ ]:
full_responses_table_name = f"""{dataset_id}.{product}_session_chunks"""
#Vismanet_AutoPay_events_filtered_L60D
query_headers = f"""SELECT
        column_name AS field_name,
        data_type AS field_type,
        description AS field_description,
    FROM
        `{dataset_id}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE
        table_name = '{product}_session_chunks';
    """

query_job = bigquery_client.query(query_headers)
table = query_job.result()

rows = list(table)

all_fields_type = {row[0]: row[1] for row in rows}
all_fields_desc = {row[0]: row[2] for row in rows}
all_fields_names = {row[0]: row[0] for row in rows}

In [ ]:
print(all_fields_desc)

In [ ]:
summary_list = {}
for i in table_names:
    query_tables_summary = f"""SELECT
        *
    FROM
        `{dataset_id}.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE
        option_name = 'description'
        AND table_name = '{i}';
    """
    query_job = bigquery_client.query(query_tables_summary)
    table = query_job.result()
    rows = list(table)
    summary_list[f"{i}"] = rows[0][5]

In [ ]:
print(summary_list)

In [ ]:
def choose_table(bigquery_client, prompt, keywords, expected, assumptions = None):
    text_with_headers = f"""

    **Prompt:**
    - {prompt}

    **Expected result:**
    - {expected}

    **Assumptions about prompt:**
    - {assumptions}

    **Table schema:**
    - Name of fields: {all_fields_names}
    - Data type of values in the fields: {all_fields_type}
    - Field descriptions: {all_fields_desc}

    **Instructions:**
    1. Identify and list all relevant fields from the table schema that are useful for answering the prompt and aligning with the expected result.
    2. Use these keywords to find the relevant fields: {keywords}.
    3. Do not include any extraneous symbols, such as JSON formatting, brackets, or custom fields.
    4. For each selected field, explain its relevance and how it helps to answer the prompt.
    5. Make use of assumptions if they exist. The assumptions are correct and is incorporated into the prompt.

    **Important:**
    - If **all necessary fields** to answer the prompt are present in the schema, proceed to list the fields and their corresponding use cases: field_name, [use case]
    - If the prompt mentions a clearly defined concept or event that is missing AND cannot be calculated or derived from the available data, return only: `False, [explanation]`
    - Do **not** make assumptions or workarounds unless explicitly allowed.

    Output format:
    field_name1, use case
    field_name2, use case
    ...

    Do **not** hallucinate or infer information.

    """
    fields_response = generate_response(
        text_with_headers, 
        temperature=0.05, 
        top_k=30, 
        top_p=0.3
    )

    print(fields_response)

In [ ]:
def field_selection(bigquery_client, prompt, keywords, expected, assumptions = None):
    print("Bigquery start.")
    #query_job = bigquery_client.query(query_headers)
    #table = query_job.result()
    # headers_values = []
    #for row in table:
    #    headers_values.append(row[:])
    text_with_headers = f"""

    **Prompt:**
    - {prompt}

    **Expected result:**
    - {expected}

    **Assumptions about prompt:**
    - {assumptions}

    **Table schema:**
    - Name of fields: {all_fields_names}
    - Data type of values in the fields: {all_fields_type}
    - Field descriptions: {all_fields_desc}

    **Instructions:**
    1. Identify and list all relevant fields from the table schema that are useful for answering the prompt and aligning with the expected result.
    2. Use these keywords to find the relevant fields: {keywords}.
    3. Do not include any extraneous symbols, such as JSON formatting, brackets, or custom fields.
    4. For each selected field, explain its relevance and how it helps to answer the prompt.
    5. Make use of assumptions if they exist. The assumptions are correct and is incorporated into the prompt.

    **Important:**
    - If **all necessary fields** to answer the prompt are present in the schema, proceed to list the fields and their corresponding use cases: field_name, [use case]
    - If the prompt mentions a clearly defined concept or event that is missing AND cannot be calculated or derived from the available data, return only: `False, [explanation]`
    - Do **not** make assumptions or workarounds unless explicitly allowed.

    Output format:
    field_name1, use case
    field_name2, use case
    ...

    Do **not** hallucinate or infer information.

    """

    print("Bigquery end.")
    fields_response = generate_response(
        text_with_headers, 
        temperature=0.05, 
        top_k=30, 
        top_p=0.3
    )

    print("----------------------------------------------------------------------------------")
    print(fields_response)


    lines = fields_response.split("\n")
    first_part = []
    second_part = []
    var_type = []


    for line in lines:
        if "," in line:
            first, second = line.split(",", 1) 
            first_part.append(first.strip())  
            second_part.append(second.strip())  

    for i in first_part:
        var_type.append(all_fields_type[i])

    fields_type = {
        first_part[i]: {
            'data_type': var_type[i],
        }
        for i in range(len(first_part))
    }

    fields_desc = {
        first_part[i]: {
            'description': second_part[i],
        }
        for i in range(len(first_part))
    }

    return fields_type, fields_desc

In [ ]:
def create_sql_code(prompt, fields_type, fields_desc, strategy, keywords, expected_result, assumptions = None):
    text_with_headers = f""""
    **Prompt:** {prompt}

    **Table schema:** 
    - Data types: {fields_type} 
    - Descriptions: {fields_desc} 
    - Table: {full_responses_table_name}

    **Strategy:**
    - {strategy} 

    **Keywords:** 
    - {keywords}

    **Expected result:** 
    - {expected_result}

    **Assumptions about prompt:** 
    - {assumptions}

    **Constraints:** 
    - The query is sent to BigQuery. It needs to follow BigQuery syntax.
    - Fully follow the strategy
    - Keep query costs as low as possible

    **Query Guidelines:**
    - Select only necessary fields; avoid extra data.
    - Show more than one entry.
    - Use accurate column names; no transformations.
    - Prefer partition filtering over `EXTRACT()`. Use `DATE_TRUNC(date_column, DATE_PART)` correctly (e.g., `DATE_TRUNC(session_started_date, WEEK)`), without string literals.
    - Use approximate counts where possible.
    - Consider clustering for better filtering.
    - **Exclude NULL values** (`IS NOT NULL`).
    - Make use of the assumptions. The assumptions are correct and is incorporated into the prompt.

    **Final Checks:**
    - Are only essential fields used?
    - Does the query align with the strategy?
    - Does the data table match keywords/expected results?
    - **Ensure `IS NOT NULL` is used in relevant columns.**
    - Is the query as a whole optimized for cost?
    - Do the data types of the fields match the operations you are performing on them?

    ### **Objective:**  
    - Generate the best possible BigQuery SQL query that:  
    1. Answers the prompt fully.  
    2. Produces the expected result accurately.  
    3. Follows the strategy step-by-step without deviation. Treat the strategy as a strict blueprint; implement each step exactly as described.

    Use standard SQL. The query will be sent to **BigQuery**, so DO NOT INCLUDE EXPLANATIONS. Make sure the query is VALID. Do not hallucinate.
    """


    sql_query_response = generate_response(
        text_with_headers, 
        temperature=0.1, 
        top_k=20, 
        top_p=0.2
    )

    return sql_query_response.replace("```sql", "").replace("```", "").strip()

In [ ]:
def correct_sql_error(prompt, keywords, expected_result, sql_code, error, fields_type, fields_desc, strategy):
    final_prompt = f"""
    **Prompt:**
    - {prompt}

    **Important keywords:**
    - {keywords}

    **Strategy for creating SQL code:**
    - {strategy}

    **Expected result:**
    - {expected_result}
    
    ### **Original SQL Query (Faulty):**
    {sql_code}

    ### **Error Message:** 
    {error}

    ### **Table Details:** 
    - Table name: {full_responses_table_name}
    - Field data types: {fields_type}
    - Field descriptions: {fields_desc}

    ### **Table Details:** 
    Fix the SQL query to resolve the error while adhering to the provided strategy, prompt, keywords, and expected result.

    ### **Instructions:**  
    1. Analyze the error in the error message and provide a corrected SQL query.
    2. The output should only include the SQL code.
    3. Follow the strategy. The strategy is correct to create the SQL code.
    
    ### **Output:**  
    <fixed SQL query>

    Ensure the corrected SQL follows the expected syntax and logic.
    """

    response = generate_response(
        final_prompt, 
        temperature=0.1, 
        top_k=40, 
        top_p=0.3
    )
    print(response)

    return response.replace("```sql", "").replace("```", "").strip()

In [ ]:
def verify(prompt, query, expected_result, assumptions, strategy):
    final_prompt = f"""
    ### **Prompt:** 
    {prompt}

    ### **Assumptions:** 
    {assumptions}

    ### **Strategy for creating SQL code:** 
    {strategy}

    ### **SQL Query:** 
    {query}

    ### **Expected result:** 
    {expected_result}

    ### **Guidelines:**  
    - The assumptions are verified with the user. They are equally important to the prompt and the expected result.

    ### **Task:**  
    1. Does the strategy fully align and cover all apsects of the query?
    1. Do the strategy and the query fully answer every part of the prompt? 
    2. Do the strategy and the query fully align with the expected result?
    3. Do the strategy and the query fully align with the assumptions?
    
    ### **Output format:**  
    Yes/No. explanation
    Yes/No. explanation
    Yes/No. explanation

    """

    response = generate_response(
        final_prompt, 
        temperature=0.1, 
        top_k=30, 
        top_p=0.3
    )

    lines = response.split("\n")
    first_part = []
    second_part = []
    for line in lines:
        if "." in line:  # Ensure there is a comma before splitting
            first, second = line.split(".", 1)  # Split only at the first comma
            first_part.append(first.strip())  # Store the first part (column name)
            second_part.append(second.strip())  # Store the second part (description)

    check_list = []
    for i in range(len(first_part)):
        if first_part[i] == "No":
            check_list.append(second_part[i])

    return check_list



In [ ]:
def result_interpretation(prompt, result, strategy):
    result_text = f"""
    ### **Prompt:**
    {prompt}  

    ### **Strategy for getting data:**   
    {strategy}  

    ### **Data:**   
    {result}  


    ### **Task:**  
    Analyze the provided dataset and generate a meaningful interpretation that answers the given prompt.  

    ### **Steps:**  
    1. **Answer the question directly.** Provide a clear, data-driven response.  
    2. **Extract key insights.** Identify trends, anomalies, or correlations that are not immediately obvious.  
    3. **Use external knowledge when relevant.** If applicable, explain insights using common patterns or domain knowledge.  
    4. **Focus on the overall pattern** rather than unnecessary details unless a specific outlier is crucial.  
    5. **State relevant assumptions.** Only include assumptions that impact the interpretation (avoid discussing column names, data types, or table structures).  

    ### **Guidelines:**  
    - **Be concise.** Prioritize clarity over lengthy explanations.  
    - **Use a structured format.** Keep responses readable with bullet points and section headings.  
    - **Avoid redundant details.** Don’t restate information that is already evident from the dataset.  
    - **Ignore missing table schema details.** Focus on strategy, definitions, and calculations instead.  
    - **Assistant with a friendly tone.** Speak to the user as if this is the data you have to asnwer the prompt. Make objective statements and discuss uncertainties when possible.  
    - **BigQuery.** The data is extracted from BigQuery with (U.S weekday numbering convention e.g. Sunday is 1 and Monday is 2).
    - **Use strategy for interpretation**. The strategy contains the strategy for creating SQL code to pull the data. There can be valuable information there to interpret the data.

    ### **Expected Output Format:**  
    1. **Best Answer to the Prompt** (direct response)  
    2. **Key Insights & Trends** (summary of findings)  
    3. **Relevant Assumptions** (only those impacting interpretation)  
    """


    final_answer_response = generate_response(
        result_text, 
        temperature=0.3,  # More creativity
        top_k=40, 
        top_p=0.6
    )

    return final_answer_response

In [ ]:
def extract_keywords(prompt, assumptions = None):
    final_prompt = f"""

    **Prompt:** 
    {prompt}

    **Assumptions about prompt:** 
    {assumptions}

    **Guidelines:** 
    - Extract the most important keywords from the prompt. 
    - The keywords will be given to a LLM together with the prompt so that it can better understand the prompt and what need to be answered.
    - Choose relevant words in the prompt that need to be considered rather that conjuring up new ones. Do not include redundant keywords.
    - Make use of assumptions if they exist. The assumptions are correct and are meant to clarify any ambiguous word or concept. You can think of them as addition to the prompt.

    ** Output format:** 
    <[keyword1, keyword2, ...]>
    
    """

    keywords = generate_response(
            final_prompt, 
            temperature=0.2, 
            top_k=20, 
            top_p=0.5
        )

    print(keywords)
    return keywords

In [ ]:
def expected(prompt, assumptions = None):
    final_prompt = f"""

    ### **Prompt:**   
    {prompt} 

    **Assumptions about prompt:** 
    {assumptions}

    **Guidelines:** 
    - What is the expected result of this prompt?. 
    - Assume user behavior data has been given.
    - Simply state the expected result in a concise way.
    - Make use of assumptions if they exist. The assumptions is meant to clarify any ambiguous word or concept. You can think of them as addition to the prompt. Do not change the prompt in a fundamental way.

    Output format:
    <Expected result>
    
    """
    
    response = generate_response(
            final_prompt, 
            temperature=0.2,  # Ensures consistency
            top_k=20, 
            top_p=0.5
        )
    print(response)
    return response

In [ ]:
aiAgent = DataQueryAgent(project_name, bigquery_client, generate_response)

In [ ]:

#prompt = "Where are most of our users from? "
#prompt = "Which customers have the highest engagement?"
prompt = "Which customers have the most sessions?"
prompt = "Based on the bigger customers; what days and times is the best for performing updates, maintenance?"
#prompt = "Based on the customers with the most users; what day and time is the best for performing updates, maintenance?"
#prompt = "How do system outages impact user behavior? "
#prompt = "How did the user behavior change between 30th december 2024, 31th december 2024, 1th january 2025 and 2nd january 2025? "
#prompt = "Do performance issues correlate with a drop in activity?"
#prompt = "What combination of browser and os are the worst?"
#prompt = "Based on user behavior, what will the peak activity hours be tomorrow?"
#prompt = "How many users used each product throughout january 2025?"
#prompt = "How does app performance affect user engagement and retention?"
#prompt = "Which devices or OS versions are experiencing the worst performance (slower load times, etc)? And are there any ptoential reason for it? Find sinsights by comparing with other metrics"
#prompt = "Can you see a pattern if you compare Network Latency, Page Load Fails, Page Loading Time and Device and browsers?"
#prompt = "for all windows user, what browser should we recomend between Edge or Chrome"
#prompt = "Can you calculate the DAU/WAU and WAU/MAU stickiness for the products for all months in 2024?"
prompt = "Can you calculate the WAU/MAU stickiness for the products for all weeks in 2024?"
#prompt = "Calculate the weekly stickiness (defined as the percentage of weekly active users who used the product in the previous week) for each product in our catalog for every week of the year 2024. Provide the output as a table with the following columns: Week Number (2024), Product name, and Weekly Stickiness (%)?"
#prompt = "Are there any specific customers that have huge page load times?"
#prompt = "Network latency & slow load times observed during peak hours"
#prompt = "Most common drop-off points"

#Stickiness
#USers from vestlandfylke has ued teh solution

response = aiAgent.process_prompt(prompt)
print(response)


In [ ]:
model = "text-embedding-004"
model = TextEmbeddingModel.from_pretrained(model)

input_texts = [
    TextEmbeddingInput(text="What is Vertex AI?"),
    TextEmbeddingInput(text="Machine learning in Google Cloud"),
]

# Get embeddings
embeddings = model.get_embeddings(input_texts)

# Display results
for text, embedding in zip(input_texts, embeddings):
    print(f"Text: {text.text}")
    print(f"Embedding (first 5 values): {embedding.values[:5]}")

In [ ]:
class DataQueryAgent:
    """
    A class responsible for processing user prompts to generate and validate SQL queries 
    using BigQuery, incorporating user clarifications and query refinement.
    """
    
    def __init__(self, project_id: str, bigquery_client, generate_response_fn):
        """Initializes the DataQueryAgent with the BigQuery client and response generator."""
        self.client = bigquery_client
        self.project_id = project_id
        self.generate_response = generate_response_fn
    
    def process_prompt(self, user_input):
        """Processes user input to generate, validate, and execute an SQL query."""
        expected_result = expected(user_input)
        keywords = extract_keywords(user_input)
        fields_type, fields_desc = field_selection(self.client, user_input, keywords, expected_result)
        assumptions = self.make_assumptions(user_input, fields_type, fields_desc, keywords, expected_result)
        while True:
            user_clarifications = input(self.clarify_assumptions(assumptions))
            if (user_clarifications == "" or user_clarifications == "yes" or user_clarifications == "y"):
                break
            assumptions = self.interpret_clarification(user_input, assumptions, user_clarifications)
        print(assumptions)
        expected_result = expected(user_input, assumptions)
        keywords = extract_keywords(user_input, assumptions)
        fields_type, fields_desc = field_selection(self.client, user_input, keywords, expected_result, assumptions)

        # Query generation and validation loop
        query_attempts = 0
        while query_attempts < 3:
            strategy = self.aggregation_strategy(user_input, fields_type, fields_desc, keywords, expected_result, assumptions)
            sql_code = create_sql_code(user_input, fields_type, fields_desc, strategy, keywords, expected_result, assumptions)
            print(sql_code)
            validation_errors = verify(user_input, sql_code, expected_result, assumptions, strategy)
            print(validation_errors)
            
            if not validation_errors:
                break  # Exit loop if query is valid
            query_attempts += 1
        else:
            return "Failed to generate a valid SQL query after 3 attempts."
        
        # Validate SQL using BigQuery dry run
        valid_result = self.validate_sql_query(user_input, keywords, expected_result, sql_code, fields_type, fields_desc, strategy)
        if not valid_result[0]:
            return "An error occurred. Please try again."
        
        # Execute query and return results
        result = valid_result[1].map(lambda x: "Missing Data" if x == "" else x)
        result = result.map(lambda x: "Missing Data" if x == "None" else x)

        interpretation = result_interpretation(user_input, result, strategy)
        print(result)
        return interpretation
    
    def validate_sql_query(self, user_input, keywords, expected, sql_code, fields_type, fields_desc, strategy):
        """Performs a dry run of the SQL query to check validity."""
        job_config = bigquery.QueryJobConfig(dry_run=True)
        for attempt in range(3):
            try:
                query_job = self.client.query(sql_code, job_config=job_config)
                query_job.result()
                print(f"Query is valid. Estimated processing: {query_job.total_bytes_processed / 1e9:.2f} GB.")
                result = self.client.query(sql_code).to_dataframe()
                return True, result
            except Exception as e:
                print(f"Query validation failed (Attempt {attempt + 1}/3): {e}")
                sql_code = correct_sql_error(user_input, keywords, expected, sql_code, e, fields_type, fields_desc, strategy)
        return False, False
    
    def interpret_clarification(self, prompt, assumptions, user_clarifications):
        """Refines assumptions based on user clarifications."""
        clarification_prompt = f"""
        ### **Initial Prompt:** {prompt}
        ### **Current Assumptions:** {assumptions}
        ### **User Clarifications:** {user_clarifications}
        
        ### **Task:**
        - Update current assumptions based on user clarifications.
        - If no new information is provided, simply return the current assumptions.
        
        ### **Output Format:**
        1. ...
        2. ...
        ...
        """
        return self.generate_response(clarification_prompt, temperature=0.2, top_k=40, top_p=0.5)
    
    def clarify_assumptions(self, assumptions):
        """Generates a message to ask the user for assumption clarifications."""
        return f"Are the following assumptions correct?\n\n{assumptions}"
    
    def make_assumptions(self, prompt, fields_type, fields_desc, keywords, expected_result):
        """Determines initial SQL strategy assumptions based on user input."""
        strategy_prompt = f"""
        ### **Prompt:** {prompt}
        ### **Keywords:** {keywords}
        ### **Expected Result:** {expected_result}
        ### **Table Info:**
        - Fields Type: {fields_type}
        - Fields Description: {fields_desc}
        
        ### **Constraints:**
        - Use only given fields.
        - No external data or assumptions beyond the provided fields.
        
        ### **Task:**
        - Propose short, reasonable, and user-friendly assumptions to resolve these ambiguities, leveraging the table info implicitly.
        - Base assumptions on the most likely intent of the prompt, keywords, and expected result, informed by available data patterns.
        - Keep assumptions simple and verifiable by the user, who lacks table schema knowledge.
        - Make yours assumptions as short and punctual as possible.
        - Do NOT generate SQL code.
        
        ### **Output Format:**
        1. ...
        2. ...
        """
        return self.generate_response(strategy_prompt, temperature=0.1, top_k=20, top_p=0.3)
    
    def aggregation_strategy(self, prompt, fields_type, fields_desc, keywords, expected_result, assumptions):
        
        final_prompt = f"""Craft a precise SQL query strategy to address the prompt, ensuring alignment with the expected results and keywords.

        ### **Prompt:**  
        - {prompt}

        ### **Keywords:**  
        - {keywords}  

        ### **Expected Result:**  
        - {expected_result}  

        ### **Table Details:**  
        - Table name: {full_responses_table_name}  
        - Field data types: {fields_type}  
        - Field descriptions: {fields_desc}  

        ### **Assumptions:**  
        - {assumptions}  

        ### **Constraints:**  
        - Use only the fields in the specified table—no external data or tables.  
        - Base the strategy solely on the given prompt, keywords, expected result, and table details.  

        ### **Guidelines:**  
        1. **Analyze the Prompt:** Break down the prompt to identify the core question or insight (e.g., trends, counts, comparisons). Use keywords to pinpoint critical elements.  
        2. **Map Fields to Prompt:** Select relevant fields from the table that directly address the prompt and expected result. Justify each field’s inclusion based on its description and data type.  
        3. **Determine Analysis Type:** Decide if aggregation (e.g., SUM, COUNT, AVG) or per-record analysis is needed, based on the expected result. Avoid unnecessary complexity.  
        4. **Address Challenges:** Identify potential issues (e.g., ambiguous terms, data limitations) and propose concise solutions using the given fields and assumptions.  
        5. **Incorporate Assumptions:** Apply the assumptions as foundational rules to shape the query logic.  

        ### **SQL Strategy Requirements:**  
        - Suggest specific SQL operations (e.g., SELECT, WHERE, GROUP BY) tailored to the prompt and expected result.  
        - Use filters, joins (within the table), or calculations only if supported by the fields and prompt.  
        - Avoid speculative insights beyond the data in the table.  
        - Present a clear, step-by-step plan using SQL concepts—do not write the full query.  

        ### **Output Format:**  
        - **SQL Strategy:**  
        - Step 1: ("Purpose" - SQL action, e.g., "Filter data" - Use WHERE clause with condition X)  
        - Step 2: ("Purpose" - SQL action, e.g., "Aggregate results" - Use COUNT on field Y)  
        - (Continue as needed)  
        - **Verification:** Confirm the strategy produces a result matching the expected result and reflects the keywords.  

        ### **Notes:**  
        - Keep the strategy concise and focused—no fluff or extraneous steps.  
        - Treat keywords as a guide to emphasize the prompt’s priorities.  
        - Do not generate executable SQL code—focus on the conceptual approach.  
        """

        sql_strategy_response = generate_response(
            final_prompt, 
            temperature=0.2, 
            top_k=40, 
            top_p=0.5
        )
        print(sql_strategy_response)

        return sql_strategy_response